In [1]:
import sys
sys.path.append('..')

from root import ROOT
from data.fwf_dataset import FwfDataset

from omegaconf import OmegaConf
import os
import json
import numpy as np
import pandas as pd

# assemble config
# cfg = OmegaConf.load(os.path.join(ROOT,'config/default.yaml'))
# cfg.general.root =  ROOT
# cfg = OmegaConf.merge(cfg, OmegaConf.load(cfg.data.split))
# with open(os.path.join(cfg.data.dataset_root, 'class_dict.json'),'r') as f:
#     cfg = OmegaConf.merge(cfg, OmegaConf.create({'data':{'label_schema':json.load(f)}}))


# print(OmegaConf.to_yaml(cfg))

In [36]:
pred.max()

11

In [37]:
gt.max()

11

In [2]:
from plyfile import PlyData, PlyElement
from utils.metrics import simple_metrics

# validate with 3-rd party libraries
import torch
from torchmetrics.classification import MulticlassJaccardIndex, MulticlassAccuracy
from sklearn.metrics import jaccard_score

batch = 'batch_5'
from glob import glob

all_metrics = dict()
mious = dict()
maccs = dict()
for exp_fp in glob(os.path.join(ROOT,f'exp/{batch}/*_v*')):
    exp_name = os.path.basename(exp_fp)
    cfg = OmegaConf.load(os.path.join(exp_fp, 'config.yaml'))
    num_classes = len(cfg.data.label_schema[cfg.data.label_names[0]])
    print(exp_name)
    # load point clouds

    gts = []
    preds = []
    for pcd_fp in glob(os.path.join(exp_fp,'test/*.ply')):
        # aggregate labels from all point clodus
        pcd_name = os.path.basename(pcd_fp)
        print(f'\t{pcd_name}')
        pcd = pd.DataFrame.from_dict(PlyData.read(pcd_fp).elements[0].data)
        gt = pcd['gt_labels_3'].to_numpy().astype(np.uint8)
        pred = pcd['pr_labels_3'].to_numpy().astype(np.uint8)
        gts.append(gt)
        preds.append(pred)
        

    # check metrics
    # miou_own = simple_metrics(gt, pred, num_classes)['miou']
    # miou_skl = jaccard_score(gt, pred, average='macro')
    gt = np.concatenate(gts)
    pred = np.concatenate(preds)
    miou_metric = MulticlassJaccardIndex(num_classes=num_classes, average=None)
    iou_pytorch = miou_metric(torch.tensor(gt), torch.tensor(pred)).numpy()
    # print(f"\t\t{miou_own=}\n\t\t{miou_skl=}\n\t\t{miou_pytorch=}")
    all_metrics[exp_name] = iou_pytorch

    miou = MulticlassJaccardIndex(num_classes=num_classes, average='macro')
    macc = MulticlassAccuracy(num_classes=num_classes,average='macro')
    mious[exp_name] = miou(torch.tensor(gt), torch.tensor(pred)).numpy()
    maccs[exp_name] = macc(torch.tensor(gt), torch.tensor(pred)).numpy()
        
    # raise Exception




2024-12-22_14-13-27_v503
	2023-08-28_FW_EingangBauing.FwfProj--defaultBbox.ply
	2024-04-05_FW_Westbahnhof_01.FwfProj--defaultBbox.ply
	2024-07-31_FW_Bruecke_Koenigstr.FwfProj--bboxId=001.ply
	2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj--bboxId=000.ply
	2024-07-31_FW_Bruecke_Turmstr.FwfProj--defaultBbox.ply
2024-12-22_14-48-44_v504
	2023-08-28_FW_EingangBauing.FwfProj--defaultBbox.ply
	2024-04-05_FW_Westbahnhof_01.FwfProj--defaultBbox.ply
	2024-07-31_FW_Bruecke_Koenigstr.FwfProj--bboxId=001.ply
	2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj--bboxId=000.ply
	2024-07-31_FW_Bruecke_Turmstr.FwfProj--defaultBbox.ply
2024-12-22_15-22-21_v506
	2023-08-28_FW_EingangBauing.FwfProj--defaultBbox.ply
	2024-04-05_FW_Westbahnhof_01.FwfProj--defaultBbox.ply
	2024-07-31_FW_Bruecke_Koenigstr.FwfProj--bboxId=001.ply
	2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj--bboxId=000.ply
	2024-07-31_FW_Bruecke_Turmstr.FwfProj--defaultBbox.ply
2024-12-22_16-03-25_v507
	2023-08-28_FW_EingangBauing.FwfProj--defaultBbox.p

In [3]:
names = pd.Series(mious.keys())
df_new = pd.DataFrame({'names': names,
                       'mious': [mious[n]*100 for n in names],
                       'maccs': [maccs[n]*100 for n in names]
                       })
df_new['names'] = df_new['names'].apply(lambda x: int(x.split('_')[-1].replace('v',''))-500)
df_new = df_new.sort_values(by='names').reset_index(drop=True)
df_new


,names,mious,maccs
0,3,21.469396,31.685710
1,4,16.925365,28.996024
2,6,26.008296,35.678291
3,7,0.304749,7.427704


In [135]:
names = pd.Series(mious.keys())
df_new = pd.DataFrame({'names': names,
                       'mious': [mious[n]*100 for n in names],
                       'maccs': [maccs[n]*100 for n in names]
                       })
df_new['names'] = df_new['names'].apply(lambda x: int(x.split('_')[-1].replace('v',''))-500)
df_new = df_new.sort_values(by='names').reset_index(drop=True)
df_new


,names,mious,maccs
0,3,21.548781,31.609440
1,4,16.629535,28.291851
2,6,22.655812,34.151623
3,7,0.265808,3.878984


In [ ]:
0 & 0.17 & 3.26 \\
1 & 10.15 & 16.64 \\
2 & 17.30 & 24.24 \\
3 & 18.04 & 27.71 \\
4 & 11.04 & 20.14 \\
5 & 13.20 & 22.42 \\
6 & 27.74 & 40.89 \\
7 & 0.49 & 3.91 \\
8 & 13.51 & 25.53 \\

In [128]:
print(df_new.to_latex(index=False, float_format="%.2f"))

\begin{tabular}{rrr}
\toprule
names & mious & maccs \\
\midrule
0 & 0.17 & 3.26 \\
1 & 10.15 & 16.64 \\
2 & 17.30 & 24.24 \\
3 & 18.04 & 27.71 \\
4 & 11.04 & 20.14 \\
5 & 13.20 & 22.42 \\
6 & 27.74 & 40.89 \\
7 & 0.49 & 3.91 \\
8 & 13.51 & 25.53 \\
\bottomrule
\end{tabular}



In [63]:
all_metrics_average = {(k,np.mean(v)) for k,v in all_metrics.items()}

In [107]:
cat_names = list(cfg.data.label_schema['labels_3'].keys())
all_metrics_df = pd.DataFrame(columns=['exp_name']+ cat_names)
all_metrics_df

for exp_name, metrics in all_metrics.items():
    # all_metrics_df['exp_name'] = exp_name
    # all_metrics_df[cat_names] = metrics
    exp_metrics = pd.DataFrame([{**{'exp_name': exp_name.split('_')[-1]}, **{n: m*100 for n, m in zip(cat_names, metrics)}}])
    all_metrics_df = pd.concat([all_metrics_df, exp_metrics], axis=0, ignore_index=True)
    # exp_metrics[exp_metrics] = exp_name

all_metrics_df.sort_values(by='exp_name')

/tmp/ipykernel_3736963/3240859199.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_metrics_df = pd.concat([all_metrics_df, exp_metrics], axis=0, ignore_index=True)


,exp_name,_unspecified,asphalt,brick,cable,concrete,marking,mesh,metal,naturalStone,poster,treeTrunk,vegetation
0,v100,0.000000,0.000117,0.000000,0.000000,0.303271,0.000000,0.000000,0.000000,0.0,1.747429,0.000000,0.000000
1,v101,0.000000,25.157344,0.000000,0.000000,38.499063,0.000000,1.065015,0.000040,0.0,9.516048,2.347692,45.195577
2,v102,8.288890,22.737588,0.000000,0.435886,39.249185,2.315074,62.257034,0.442449,0.0,17.925505,4.302786,49.657127
3,v103,11.111057,27.667335,0.000000,4.785065,34.313387,4.915421,55.282044,4.514446,0.0,12.074471,4.246407,57.596701
4,v104,6.105175,28.170350,0.000000,0.009592,27.952585,4.173307,18.332043,3.437573,0.0,7.009216,4.265192,33.043504
7,v105,4.544444,19.791049,0.003301,0.019152,38.314477,1.884449,16.754295,12.923151,0.0,16.793825,4.008681,43.398407
8,v106,9.234324,48.791528,0.013593,19.357683,48.367980,14.985089,63.172591,37.517339,0.0,13.430773,9.026773,68.940192
5,v107,0.000000,0.007866,0.000000,0.000000,4.107670,0.000000,0.000000,0.000000,0.0,1.749402,0.000000,0.000000
6,v108,0.022143,51.528925,0.000000,0.005493,33.757877,0.000000,0.060415,0.234122,0.0,13.160144,1.425830,61.932719


In [108]:
print(all_metrics_df.to_latex(index=False, float_format="%.2f"))

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
exp_name & _unspecified & asphalt & brick & cable & concrete & marking & mesh & metal & naturalStone & poster & treeTrunk & vegetation \\
\midrule
v100 & 0.00 & 0.00 & 0.00 & 0.00 & 0.30 & 0.00 & 0.00 & 0.00 & 0.00 & 1.75 & 0.00 & 0.00 \\
v101 & 0.00 & 25.16 & 0.00 & 0.00 & 38.50 & 0.00 & 1.07 & 0.00 & 0.00 & 9.52 & 2.35 & 45.20 \\
v102 & 8.29 & 22.74 & 0.00 & 0.44 & 39.25 & 2.32 & 62.26 & 0.44 & 0.00 & 17.93 & 4.30 & 49.66 \\
v103 & 11.11 & 27.67 & 0.00 & 4.79 & 34.31 & 4.92 & 55.28 & 4.51 & 0.00 & 12.07 & 4.25 & 57.60 \\
v104 & 6.11 & 28.17 & 0.00 & 0.01 & 27.95 & 4.17 & 18.33 & 3.44 & 0.00 & 7.01 & 4.27 & 33.04 \\
v107 & 0.00 & 0.01 & 0.00 & 0.00 & 4.11 & 0.00 & 0.00 & 0.00 & 0.00 & 1.75 & 0.00 & 0.00 \\
v108 & 0.02 & 51.53 & 0.00 & 0.01 & 33.76 & 0.00 & 0.06 & 0.23 & 0.00 & 13.16 & 1.43 & 61.93 \\
v105 & 4.54 & 19.79 & 0.00 & 0.02 & 38.31 & 1.88 & 16.75 & 12.92 & 0.00 & 16.79 & 4.01 & 43.40 \\
v106 & 9.23 & 48.79 & 0.01 & 19.36 & 48.37 & 14.9

In [74]:
all_metrics_str = {k:[f'{v*100:.3f}' for v in vv] for k, vv in all_metrics.items()}
all_metrics_str

{'2024-12-18_01-00-54_v100': ['0.000',
  '0.000',
  '0.000',
  '0.000',
  '0.303',
  '0.000',
  '0.000',
  '0.000',
  '0.000',
  '1.747',
  '0.000',
  '0.000'],
 '2024-12-18_01-32-40_v101': ['0.000',
  '25.157',
  '0.000',
  '0.000',
  '38.499',
  '0.000',
  '1.065',
  '0.000',
  '0.000',
  '9.516',
  '2.348',
  '45.196'],
 '2024-12-18_02-19-55_v102': ['8.289',
  '22.738',
  '0.000',
  '0.436',
  '39.249',
  '2.315',
  '62.257',
  '0.442',
  '0.000',
  '17.926',
  '4.303',
  '49.657'],
 '2024-12-18_03-13-01_v103': ['11.111',
  '27.667',
  '0.000',
  '4.785',
  '34.313',
  '4.915',
  '55.282',
  '4.514',
  '0.000',
  '12.074',
  '4.246',
  '57.597'],
 '2024-12-18_03-51-56_v104': ['6.105',
  '28.170',
  '0.000',
  '0.010',
  '27.953',
  '4.173',
  '18.332',
  '3.438',
  '0.000',
  '7.009',
  '4.265',
  '33.044'],
 '2024-12-18_04-30-09_v107': ['0.000',
  '0.008',
  '0.000',
  '0.000',
  '4.108',
  '0.000',
  '0.000',
  '0.000',
  '0.000',
  '1.749',
  '0.000',
  '0.000'],
 '2024-12-18_05-

In [110]:
for k,v in all_metrics_average:
    print(k.split('_')[-1], f'{v*100:.2f}')

v107 0.49
v105 13.20
v108 13.51
v101 10.15
v100 0.17
v106 27.74
v104 11.04
v102 17.30
v103 18.04


In [41]:
num_classes = len(cfg.data.label_schema[cfg.data.label_names[0]])
simple_metrics(gt, pred, num_classes)

{'iou': array([0.00000000e+00, 2.34436906e-06, 0.00000000e+00,            nan,
        3.05010836e-03,            nan, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00,            nan, 0.00000000e+00, 0.00000000e+00]),
 'pc_acc': array([0.00000000e+00, 2.34437456e-06, 0.00000000e+00,            nan,
        3.05943999e-03,            nan, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 'miou': 0.00033916141462342017,
 'macc': 0.0003061784360597011}

In [58]:
miou_metric = MulticlassJaccardIndex(num_classes=num_classes, average="none")

miou_metric(torch.tensor(gt),torch.tensor(pred))



tensor([1.2990e-01, 4.3901e-01, 0.0000e+00, 2.7946e-04, 1.3021e-01, 7.2557e-02,
        4.9684e-01, 3.5172e-02, 0.0000e+00, 2.1525e-01, 0.0000e+00, 3.9165e-02])